In [1]:
import mlflow

# Start an MLflow experiment
mlflow.set_experiment('GridSearchCV_Experiment')

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2023/11/14 18:48:36 INFO mlflow.tracking.fluent: Experiment with name 'GridSearchCV_Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/P5/mlruns/992333753801301528', creation_time=1699984116641, experiment_id='992333753801301528', last_update_time=1699984116641, lifecycle_stage='active', name='GridSearchCV_Experiment', tags={}>

In [2]:
import os
import ast
import numpy as np
import pandas as pd


ROOT_PATH = os.getcwd()
DATA_LOAD_FILE = os.path.join(ROOT_PATH, "data/StackOverFlowEmbedded.csv.gz")

df = pd.read_csv(DATA_LOAD_FILE)
df["BertEmbeddings"] = df["BertEmbeddings"].apply(ast.literal_eval)

#df["FastTextEmbeddings"] = df["FastTextEmbeddings"].apply(ast.literal_eval)
#df["UseEmbeddings"] = df["UseEmbeddings"].apply(ast.literal_eval)
df

,Date,Title,Tags,Score,Label,TitleClean,TitleCleanTokenised,BertEmbeddings,FastTextEmbeddings,UseEmbeddings
0,2013-12-10 14:33:34,document.querySelector(...) is null error,javascript,66,javascript,documentqueryselector is null error,"['documentqueryselector', 'is', 'null', 'error']","[-0.34739574790000916, -0.21379917860031128, 0...",[-0.02552597 -0.1010939 0.19344184 0.063939...,"[-0.015874912962317467, -0.04766062647104263, ..."
1,2016-01-28 05:57:25,How to set activeClassName for wrapper element...,"javascript, reactjs, react-router, redux",50,javascript,how to set activeclassnam for wrapper element ...,"['how', 'to', 'set', 'activeclassnam', 'for', ...","[-0.3783631920814514, -0.4589630365371704, 0.0...",[-0.02221784 -0.10171874 0.18126324 0.061942...,"[0.011294362135231495, -0.03345678001642227, 0..."
2,2012-05-23 10:01:29,Javascript decoding html entities,"javascript, jquery",64,javascript,javascript decod html entiti,"['javascript', 'decod', 'html', 'entiti']","[0.1057375818490982, -0.2604887783527374, 0.13...",[-0.02405 -0.1134114 0.2044202 0.070119...,"[-0.009631449356675148, -0.018550606444478035,..."
3,2013-05-26 01:50:13,check if function is a generator,"javascript, generator, yield, ecmascript-6",68,javascript,check if function is a gener,"['check', 'if', 'function', 'is', 'a', 'gener']","[-0.10197962820529938, -0.4015989303588867, -0...",[-0.02910706 -0.09673091 0.18132402 0.055359...,"[-0.04886353760957718, -0.06259043514728546, 0..."
4,2012-05-15 13:40:52,Convert long number into abbreviated string in...,"javascript, numbers",120,javascript,convert long number into abbrevi string in jav...,"['convert', 'long', 'number', 'into', 'abbrevi...","[-0.06461355090141296, -0.4779772460460663, -0...",[-0.02134252 -0.10422874 0.17935485 0.060461...,"[-0.04029103368520737, -0.08340628445148468, 0..."
...,...,...,...,...,...,...,...,...,...,...
1995,2014-10-08 23:00:19,How do I count the NaN values in a column in p...,"python, pandas, dataframe",785,python,how do i count the nan valu in a column in pan...,"['how', 'do', 'i', 'count', 'the', 'nan', 'val...","[-0.055721115320920944, -0.09026826918125153, ...",[-0.02050154 -0.10144608 0.17274226 0.059439...,"[-0.011467344127595425, -0.08926451206207275, ..."
1996,2010-06-29 14:54:07,how to delete files from amazon s3 bucket?,"python, amazon-s3, bucket",165,python,how to delet file from amazon s3 bucket,"['how', 'to', 'delet', 'file', 'from', 'amazon...","[-0.1046925038099289, -0.46713075041770935, -0...",[-0.017755 -0.09092246 0.16013071 0.056149...,"[-0.04195282980799675, 0.042264651507139206, 0..."
1997,2013-06-04 18:46:56,Writing a pandas DataFrame to CSV file,"python, csv, pandas, dataframe",1091,python,write a panda datafram to csv file,"['write', 'a', 'panda', 'datafram', 'to', 'csv...","[-0.12554359436035156, -0.27312299609184265, 0...",[-0.01945207 -0.10043164 0.17147835 0.058537...,"[-0.02797633223235607, -0.07273181527853012, 0..."
1998,2013-02-08 11:41:57,How do I change the figure size with subplots?,"python, matplotlib, subplot, figure",640,python,how do i chang the figur size with subplot,"['how', 'do', 'i', 'chang', 'the', 'figur', 's...","[-0.1792479008436203, -0.07812405377626419, 0....",[-0.01877434 -0.09718987 0.1683856 0.060309...,"[0.0023067439906299114, -0.06395777314901352, ..."


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
df["LabelEncoded"] = LabelEncoder().fit_transform(df["Label"].values)
TrainDF, TestDF = train_test_split(df,train_size=0.7, random_state=42, stratify=df["LabelEncoded"].values)

In [5]:

with mlflow.start_run():
    logistic_regression_model = LogisticRegression()
    parameter_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': [None, 'l2']}
    grid_search = GridSearchCV(estimator=logistic_regression_model,
                            param_grid=parameter_grid,
                            scoring='accuracy',
                            cv=10)
    grid_search.fit(np.array(TrainDF["BertEmbeddings"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    for param_name in best_params:
        mlflow.log_param(param_name, best_params[param_name])
    mlflow.log_metric("best_score", grid_search.best_score_)

    y_pred = best_model.predict(np.array(TestDF["BertEmbeddings"].values.tolist()))
    accuracy = accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), y_pred)
    print(f"Accuracy of the best model: {accuracy}")

    # Log the best model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "best_model")

    # End the MLflow run
    mlflow.end_run()

/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/.venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/typha

Accuracy of the best model: 0.805


In [7]:
#!mlflow ui

In [14]:
random_forest_model = RandomForestClassifier()
parameter_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=random_forest_model,
                           param_grid=parameter_grid,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1)  # n_jobs=-1 to use all processors
grid_search.fit(np.array(TrainDF["BertEmbeddings"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

best_model = grid_search.best_estimator_
y_pred = best_model.predict(np.array(TestDF["BertEmbeddings"].values.tolist()))
accuracy = accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), y_pred)
print(f"Accuracy of the best model: {accuracy}")

Accuracy of the best model: 0.7533333333333333


In [17]:
decision_tree_model = DecisionTreeClassifier()
parameter_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=decision_tree_model,
                           param_grid=parameter_grid,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1)  # n_jobs=-1 to use all processors
grid_search.fit(np.array(TrainDF["BertEmbeddings"].values.tolist()), np.array(TrainDF["LabelEncoded"].values.tolist()))

best_model = grid_search.best_estimator_
y_pred = best_model.predict(np.array(TestDF["BertEmbeddings"].values.tolist()))
accuracy = accuracy_score(np.array(TestDF["LabelEncoded"].values.tolist()), y_pred)
print(f"Accuracy of the best model: {accuracy}")

Accuracy of the best model: 0.645
